In [1]:
import numpy as np
import pandas as pd

import os
import time
import datetime

from redshiftpool import RedShift
from mysqlpool import Mysql

In [3]:
redshift = RedShift()
#mysql = Mysql()

create connect redshift
create connect


In [4]:
sql_train_neg = "select c.token,to_char(client_time,'YYYY-MM-DD HH24:MI:SS'),c.sta_key,to_char(create_time,'YYYY-MM-DD HH24:MI:SS') \
from (select b.* from (select token from sta_new_user where server_time>='2017-06-01' \
and server_time<'2017-06-06' and product_id=600027) a ,(select * from (select token,client_time,sta_key,ROW_NUMBER () OVER (PARTITION BY token order by client_time asc) as row \
from sta_event_game_publish where server_time>='2017-06-01' and server_time<'2017-06-11' and product_id=600027 \
and substring(sta_key,1,1)='T') t where row<=30) b where a.token=b.token) c left join (select token,min(create_time) as create_time \
from game_iap where create_time>='2017-06-01' and product_id=600027 group by token)d on c.token=d.token where d.token is null;"

sql_train_neg = "select c.token,to_char(client_time,'YYYY-MM-DD HH24:MI:SS'),c.sta_key,to_char(create_time,'YYYY-MM-DD HH24:MI:SS') \
from (select b.* from (select token from sta_new_user where server_time>='2017-07-01' \
and server_time<'2017-07-06' and product_id=600027) a ,(select * from (select token,client_time,sta_key,ROW_NUMBER () OVER (PARTITION BY token order by client_time asc) as row \
from sta_event_game_publish where server_time>='2017-07-01' and server_time<'2017-07-11' and product_id=600027 \
and substring(sta_key,1,1)='T') t where row<=30) b where a.token=b.token) c left join (select token,min(create_time) as create_time \
from game_iap where create_time>='2017-07-01' and product_id=600027 group by token)d on c.token=d.token where d.token is null;"


sql_train_pos = "select c.token,to_char(client_time,'YYYY-MM-DD HH24:MI:SS'),c.sta_key,to_char(create_time,'YYYY-MM-DD HH24:MI:SS') \
from (select b.* from (select token from sta_new_user where server_time>='2017-03-01' and product_id=600027) a ,\
(select * from (select token,client_time,sta_key,ROW_NUMBER () OVER (PARTITION BY token order by client_time asc) as row \
from sta_event_game_publish where server_time>='2017-03-01' and product_id=600027 \
and substring(sta_key,1,1)='T') t where row<=30) b where a.token=b.token) c join (select token,min(create_time) as create_time \
from game_iap where create_time>='2017-04-01' and product_id=600027 group by token)d on c.token=d.token;"



In [5]:
def getRawData(s,filename):
    raw = redshift.getAll(s)
    fileObject = open(filename, 'w')  
    for r in raw:  
        fileObject.write(str(r))  
        fileObject.write('\n')  
    fileObject.close() 
    return raw

In [6]:
#raw_train_neg = getRawData(sql_train_neg,'raw_train_neg.txt')
raw_train_pos = getRawData(sql_train_pos,'raw_train_pos.txt')

In [7]:
raw_train_neg = getRawData(sql_train_neg,'raw_train_neg.txt')

In [10]:
#公共方法
def getDic(data):
    positiveDataDic={} #是一个储存正样本的字典
    negativeDataDic={} #是一个储存负样本的字典
    for i in range(len(data)):
        key=data[i][0]
        value = []
        value.extend(data[i][1:3])
        if data[i][3]:
            if positiveDataDic.has_key(key):
                positiveDataDic[key].extend([value])
            else:
                positiveDataDic[key]=[value]
        else:
            if negativeDataDic.has_key(key):
                negativeDataDic[key].extend([value])
            else:
                negativeDataDic[key]=[value]

    return positiveDataDic,negativeDataDic

In [11]:
print raw_train_neg[-10:]
dic_train_neg_pos,dic_train_neg_neg = getDic(raw_train_neg)
dic_train_pos_pos,dic_train_pos_neg = getDic(raw_train_pos)


[('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:05:38', 'T0S', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:05:38', 'T08', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:05:38', 'T0K', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:05:57', 'T07', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:06:11', 'T09', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:06:13', 'T09', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:07:02', 'T08', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:07:02', 'T0S', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:07:02', 'T0K', None), ('ffc06075dd474f3282a757bdaa5fa771', '2017-07-04 06:07:20', 'T07', None)]


In [12]:
keys=['GIB','T01','T02','T03','T04','T05','T06','T07','T08','T09','T0F','T0G','T0H','T0I','T0J','T0K','T0L','T0M','T0O','T0P','T0R','T0S','T0T','T0U','T0V','T0N']

key_indices = dict((key, i) for i, key in enumerate(keys))
indices_key = dict((i, key) for i, key in enumerate(keys))

In [13]:
def makeTrain(dic):
    result = []
    for (k,v) in dic.items():
        if len(v)<21:
            continue
        else:
            v.sort()
            value = [key_indices[v[i][1]] for i in range(1,21)]
            result.append(value)
    
    return result    

In [14]:
pos = makeTrain(dic_train_pos_pos)
neg = makeTrain(dic_train_neg_neg)

print len(pos)
print len(neg)

print pos[0:3]
print neg[0:3]

7515
12186
[[21, 1, 4, 5, 9, 9, 15, 21, 8, 23, 7, 9, 9, 8, 15, 21, 7, 9, 9, 21], [21, 1, 4, 5, 9, 9, 21, 8, 15, 2, 13, 23, 23, 7, 9, 9, 8, 15, 21, 7], [21, 1, 4, 5, 9, 9, 8, 15, 21, 23, 7, 9, 9, 8, 15, 21, 7, 9, 9, 8]]
[[21, 1, 2, 13, 23, 4, 2, 13, 16, 1, 2, 13, 23, 2, 13, 23, 5, 16, 1, 16], [21, 1, 2, 4, 13, 23, 5, 9, 9, 15, 21, 8, 2, 13, 23, 23, 7, 9, 9, 8], [2, 13, 21, 23, 1, 4, 5, 9, 9, 15, 21, 8, 23, 10, 7, 9, 9, 8, 15, 21]]


In [15]:
import random
import copy

random.shuffle(neg)

tlen = 6500

trainX = copy.deepcopy(pos[0:tlen])
trainX.extend(neg[0:tlen])
trainY = np.ones(tlen)
trainY = np.append(trainY, np.zeros(tlen))

testX = copy.deepcopy(pos[tlen:])
leng = len(testX)
testY = np.ones(leng)

print leng
print len(testX),len(testY)
testX.extend(neg[tlen:tlen+leng])
testY = np.append(testY, np.zeros(leng))

print len(trainX),len(trainY)
print len(testX),len(testY)
print testY.shape

print trainY

1015
1015 1015
13000 13000
2030 2030
(2030,)
[ 1.  1.  1. ...,  0.  0.  0.]


In [16]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers.normalization import BatchNormalization

maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

Using TensorFlow backend.


In [17]:
trainX = np.array(trainX)
testX = np.array(testX)

In [18]:
print('Build model...')
model = Sequential()
model.add(Embedding(26, 128))
model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.2))
model.add(BatchNormalization())
model.add(Dense(64,activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         3328      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 128)         512       
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65    

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=20,
          shuffle=True,
          validation_data=(testX, testY))


Train...
Train on 13000 samples, validate on 2030 samples
Epoch 1/20
13000/13000 [==============================] - 75s - loss: 0.4460 - acc: 0.7745 - val_loss: 0.3864 - val_acc: 0.7951
Epoch 2/20
13000/13000 [==============================] - 71s - loss: 0.4012 - acc: 0.7862 - val_loss: 0.3797 - val_acc: 0.8030
Epoch 3/20
13000/13000 [==============================] - 70s - loss: 0.3961 - acc: 0.7858 - val_loss: 0.3738 - val_acc: 0.7956
Epoch 4/20
13000/13000 [==============================] - 69s - loss: 0.3884 - acc: 0.7912 - val_loss: 0.3816 - val_acc: 0.8108
Epoch 5/20
13000/13000 [==============================] - 70s - loss: 0.3867 - acc: 0.7984 - val_loss: 0.3620 - val_acc: 0.8158
Epoch 6/20
13000/13000 [==============================] - 70s - loss: 0.3809 - acc: 0.8025 - val_loss: 0.3727 - val_acc: 0.8153
Epoch 7/20
13000/13000 [==============================] - 68s - loss: 0.3774 - acc: 0.8028 - val_loss: 0.3648 - val_acc: 0.8108
Epoch 8/20
13000/13000 [======================

In [20]:
import time
print time.time()
result = model.predict(trainX)
print time.time()

1499849138.53
1499849157.17


In [21]:
#验证训练数据
print len(result)
posl = result[:tlen]
negl = result[tlen:]

posloc = np.where(posl>=0.35)
negloc = np.where(negl>=0.35)
print len(posloc[0])
print len(negloc[0])


posloc = np.where(posl>=0.45)
negloc = np.where(negl>=0.45)

print len(posloc[0])
print len(negloc[0])

13000
5984
2262
5743
1746


In [22]:
print result

[[ 0.47538951]
 [ 0.51247615]
 [ 0.48711824]
 ..., 
 [ 0.48644963]
 [ 0.12136541]
 [ 0.07643696]]


In [23]:
resultpos = model.predict(np.array(pos))
resultneg = model.predict(np.array(neg))

In [28]:
rate=0.2

posloc = np.where(resultpos>=rate)
negloc = np.where(resultpos<=rate)
print len(resultpos)
print len(posloc[0])
print "分到正类百分比：",len(posloc[0])*1.0/len(resultpos)

posloc = np.where(resultneg>=rate)
negloc = np.where(resultneg<=rate)
print len(resultneg)
print len(posloc[0])
print "分到负类百分比：",len(negloc[0])*1.0/len(resultneg)

7515
7129
分到正类百分比： 0.948636061211
12186
5290
分到负类百分比： 0.565895289677


In [34]:
#推荐使用的模型
from keras.layers import Input, Embedding, LSTM, Dense,Flatten, Dropout, merge
from keras.layers import Input, Activation, Permute, Dropout, add, dot, concatenate
from keras.layers.core import Reshape
from keras.models import Model
input_1=Input(shape=(20,), dtype='int32')
input_2=Input(shape=(1,), dtype='int32')
x1=Embedding(output_dim=32, input_dim=25, input_length=20)(input_1)
x1=LSTM(128, return_sequences=True)(x1)
x1=LSTM(32)(x1)
x1=Reshape((-1,))(x1)
x2=Embedding(output_dim=32, input_dim=25, input_length=1)(input_2)
x2=Flatten()(x2)
x = concatenate([x1, x2])
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
out = Dense(1, activation='sigmoid')(x)
model = Model([input_1, input_2], out)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 20)            0                                            
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 20, 32)        800                                          
____________________________________________________________________________________________________
lstm_7 (LSTM)                    (None, 20, 128)       82432                                        
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 1)             0                                            
___________________________________________________________________________________________

In [80]:
#pytorch
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torch import nn
torch.manual_seed(1)    # reproducible

BATCH_SIZE = 32      # 批训练的数据个数


torchx = torch.from_numpy(trainX)
#torchx = Variable(torchx)
print trainY
#trainY.dtype = 'int64'
torchy = torch.from_numpy(trainY).type(torch.LongTensor)
print trainY
print torchy
torch_test_x = Variable(torch.from_numpy(testX), volatile=True) 
print torch_test_x
torch_test_y = testY.squeeze()    # covert to numpy array
print torch_test_y

#torchy = Variable(torchy)

# 先转换成 torch 能识别的 Dataset
torch_dataset = Data.TensorDataset(data_tensor=torchx, target_tensor=torchy)

# 把 dataset 放入 DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,      # torch TensorDataset format
    batch_size=BATCH_SIZE,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)


[ 1.  1.  1. ...,  0.  0.  0.]
[ 1.  1.  1. ...,  0.  0.  0.]

 1
 1
 1
⋮ 
 0
 0
 0
[torch.LongTensor of size 13000]

Variable containing:
   21     1     2  ...     23     9     2
   21     1     5  ...     23     3     7
   21     1     2  ...     13    23     8
       ...          ⋱          ...       
    1    21     4  ...      9     9    21
   21     1     2  ...      9     8    15
   21     1     4  ...      9     9     8
[torch.LongTensor of size 1752x20]

[ 1.  1.  1. ...,  0.  0.  0.]


In [87]:
class Net(torch.nn.Module):  # 继承 torch 的 Module
    def __init__(self):
        super(Net, self).__init__()     # 继承 __init__ 功能
        # 定义每层用什么样的形式
        self.embed = torch.nn.Embedding(26, 128)
        self.rnn = nn.LSTM(     # LSTM 效果要比 nn.RNN() 好多了
            input_size=128,      # 
            hidden_size=128,     # rnn hidden unit
            num_layers=2,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.hidden = torch.nn.Linear(128, 64)   # 隐藏层线性输出
        self.predict = torch.nn.Linear(64, 2)   # 输出层线性输出

    def forward(self, x):   # 这同时也是 Module 中的 forward 功能
        x = self.embed(x)
        r_out, (h_n, h_c) = self.rnn(x, None) 
        #x = F.relu(self.hidden(x))      # 激励函数(隐藏层的线性值)
        x = F.relu(self.hidden(r_out[:, -1, :]))
        x = self.predict(x)
        return x
    
rnn = Net()
print(rnn)

Net (
  (embed): Embedding(26, 128)
  (rnn): LSTM(128, 128, num_layers=2, batch_first=True)
  (hidden): Linear (128 -> 64)
  (predict): Linear (64 -> 2)
)


In [88]:
# optimizer 是训练的工具
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.5)  # 传入 net 的所有参数, 学习率
#optimizer = torch.optim.Adam(rnn.parameters(), lr=0.1) 

# 算误差的时候, 注意真实值!不是! one-hot 形式的, 而是1D Tensor, (batch,)
# 但是预测值是2D tensor (batch, n_classes)
loss_func = torch.nn.CrossEntropyLoss()

for t in range(1):
    for step, (batch_x, batch_y) in enumerate(loader):
        pass
        #print type(batch_x)
        

In [90]:
print time.time()
for t in range(20):
    for step, (batch_x, batch_y) in enumerate(loader):
        b_x = Variable(batch_x)              # reshape x to (batch, time_step, input_size)
        b_y = Variable(batch_y)                               # batch y
        #print b_x
        
        output = rnn(b_x)         # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step() 
        
        
    test_output = rnn(torch_test_x)
    #test_output = rnn(b_x)
    
    #print test_output
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    print pred_y
    accuracy = sum(pred_y == torch_test_y) / float(torch_test_y.size)
    print('Epoch: ', t, '| train loss: %.4f' % loss.data[0], '| test accuracy: %.2f' % accuracy)
print time.time()   

1499682397.8
[0 1 1 ..., 0 0 0]
('Epoch: ', 0, '| train loss: 0.3129', '| test accuracy: 0.81')
[0 1 1 ..., 0 0 0]
('Epoch: ', 1, '| train loss: 0.2392', '| test accuracy: 0.80')
[0 1 1 ..., 0 0 0]
('Epoch: ', 2, '| train loss: 0.4402', '| test accuracy: 0.82')
[0 1 1 ..., 0 0 0]
('Epoch: ', 3, '| train loss: 0.2290', '| test accuracy: 0.81')
[0 1 1 ..., 0 0 0]
('Epoch: ', 4, '| train loss: 0.2646', '| test accuracy: 0.83')
[0 1 1 ..., 0 0 0]
('Epoch: ', 5, '| train loss: 0.1874', '| test accuracy: 0.82')
[0 1 1 ..., 0 0 0]
('Epoch: ', 6, '| train loss: 0.3087', '| test accuracy: 0.83')
[0 1 1 ..., 0 0 0]
('Epoch: ', 7, '| train loss: 0.3967', '| test accuracy: 0.83')
[0 1 1 ..., 0 0 0]
('Epoch: ', 8, '| train loss: 0.4457', '| test accuracy: 0.82')
[0 1 1 ..., 0 0 0]
('Epoch: ', 9, '| train loss: 0.3595', '| test accuracy: 0.81')
[0 1 1 ..., 0 0 0]
('Epoch: ', 10, '| train loss: 0.1810', '| test accuracy: 0.82')
[0 1 1 ..., 0 0 0]
('Epoch: ', 11, '| train loss: 0.1153', '| test accura

In [57]:
test_output = rnn(b_x)

In [60]:
print test_output
print b_y
print batch_y

Variable containing:
 6.8061 -6.6220
 7.2712 -6.5124
 6.8020 -6.4809
 6.8020 -6.4809
 6.8489 -6.5292
 6.6334 -6.6031
 6.8020 -6.4809
 7.1402 -6.7817
[torch.FloatTensor of size 8x2]

Variable containing:
 0.0000e+00
 4.6072e+18
 0.0000e+00
 0.0000e+00
 4.6072e+18
 4.6072e+18
 0.0000e+00
 0.0000e+00
[torch.LongTensor of size 8]


 0.0000e+00
 4.6072e+18
 0.0000e+00
 0.0000e+00
 4.6072e+18
 4.6072e+18
 0.0000e+00
 0.0000e+00
[torch.LongTensor of size 8]

